7장 딥러닝을 위한 고급 도구
------

### 7.1 Sequential 모델을 넘어서 : 케라스의 함수형 API

지금까지 모든 신경망은 Sequential 모델을 사용하여 만듬

일부 네트워크는 개별 입력이 여러 개 필요하거나 출력이 여러 개 필요. 층을 차례대로 쌓지 않고 층 사이를 연결하여 그래프처럼 만드는 네트워크도 있다.

다중 입력 모델 예시
- 모듈 병합
  - 완전 연결 모듈 - 메타데이터
  - RNN 모듈 - 텍스트 설명
  - 컨브넷 모듈 - 사진
- 가중치를 구해서 가격 예측

최근에 개발된 많은 신경망 구조는 선형적이지 않은 네트워크 토폴로지(topology)가 필요
- 빈순환 유향 그래프 같은 네트워크 구조

![alt text](https://www.researchgate.net/profile/Bo_Zhao48/publication/312515254/figure/fig3/AS:489373281067012@1493687090916/nception-module-of-GoogLeNet-This-figure-is-from-the-original-paper-10.png)

최근에는 모델에 잔차 연결을 추가하는 경향도 있음

![alt text](https://miro.medium.com/max/6652/1*OFfO8VzLv8GNFNRKafvB7w.png)

이런 경우는 케라스의 Sequential 클래스를 사용해서는 만들지 못 함

함수형 API를 사용하여 만들 수 있음

#### 7.1.1 함수형 API 소개

함수형 API에서는 직접 텐서들의 입출력을 다룸
- 함수처럼 층을 사용하여 텐서를 입력받고 출력함 -> 그래서 함수형 API라고 부름

In [3]:
from keras import Input, layers

input_tensor = Input(shape=(32, )) # 텐서
dense = layers.Dense(32, activation='relu') # 함수처럼 사용하기 위해 층 객체를 만듬

output_tensor = dense(input_tensor) # 텐서와 함께 층을 호출하면 텐서를 반환

Sequential 모델과 함수형 API로 만든 동일한 모델을 나란히 비교

In [4]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64, )))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

# 함수형 API를 만든 모델
input_tensor = Input(shape=(64, ))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

# 입력과 출력 텐서를 지정하여 Model 클래스의 객체를 만듬
model = Model(input_tensor, output_tensor)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.compile(
    optimizer='rmsprop', loss='categorical_crossentropy'
)

import numpy as np

x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 0s 239us/step - loss: 12.2830
Epoch 2/10
1000/1000 [==============================] - 0s 14us/step - loss: 14.2161
Epoch 3/10
1000/1000 [==============================] - 0s 10us/step - loss: 17.6463
Epoch 4/10
1000/1000 [==============================] - 0s 12us/step - loss: 22.1052
Epoch 5/10
1000/1000 [==============================] - 0s 11us/step - loss: 27.7020
Epoch 6/10
1000/1000 [==============================] - 0s 11us/step - loss: 34.2840
Epoch 7/10
1000/1000 [==============================] - 0s 11us/step - loss: 41.4813
Epoch 8/10
1000/1000 [==============================] - 0s 14us/step - loss: 49.6167
Epoch 9/10
1000/1000 [==============================] - 0s 13us/step - loss: 58.7254
Epoch 10/10
1000/1000 [==============================] - 0s 9us/step - loss: 68.8574


In [6]:
score=model.evaluate(x_train, y_train)

1000/1000 [==============================] - 0s 35us/step


In [7]:
score

74.94670776367188

#### 7.1.2 다중 입력 모델

함수형 API는 다중 입력 모델을 만든느 데 사용할 수 있다ㅏ.

일반적으로 이런 모델은 서로 다른 입력 가지를 합치기 위해 여러 텐서를 연결할 수 있는 층을 사용
- 텐서를 더하거나 이어 붙이는 식

```python
keras.layers.add, keras.layers.concatenate
```

question-answering 모델을 이용해서 확인해보자
- 질문-응답 모델은 2개의 입력을 가짐
- 하나는 자연어 질문, 또 하나는 답변에 필요한 정보가 담겨 있는 텍스트(예를 들어 뉴스 기사)
- 가장 간단한 구조는 미리 정의한 어휘 사전에서 소프트맥스 함수를 통해 한 단어로 된 답을 출력


응답
- Dense
- concatenate
  - LSTM - Embedding - 참고 텍스트
  - LSTM - Embedding - 질문

In [12]:
from keras.models import Model
from keras import layers
from keras import Input

text_voca_size = 10000
question_voca_size = 10000
answer_voca_size = 500

# 텍스트 입력은 길이가 정해지지 않은 정수 시퀀스. 입력 이름을 지정할 수 있다.
text_input = Input(shape=(None, ), dtype='int32', name='text')

# 입력을 크기가 64인 벡터의 시퀀스로 임베딩
embedded_text = layers.Embedding(text_voca_size, 64)(text_input)

# LSTM을 사용하여 이 벡터들을 하나의 벡터로 인코딩
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None, ), dtype='int32', name='question')
embedded_question = layers.Embedding(question_voca_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# 인코딩된 질문과 텍스트를 연결
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_voca_size, activation='softmax')(concatenated)

# 모델 객체를 만들고 2개의 입력과 출력을 주입
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [13]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, None, 32)     320000      question[0][0]                   
____________________________________________________________________________________________

이 모델은 입력이 2개인데 어떻게 훈련을 할까? 두 가지 방법이 존재
- 넘파이 배열의 리스트를 주입
- 입력 이름과 넘파이 배열로 이루어진 딕셔너리를 모델의 입력으로 주입

In [15]:
import numpy as np
from keras.utils import to_categorical

num_samples = 1000
max_length = 100

# 랜덤한 넘파이 데이터를 생성
text = np.random.randint(1, text_voca_size, size=(num_samples, max_length))
question = np.random.randint(1, question_voca_size, size=(num_samples, max_length))

answers = np.random.randint(0, answer_voca_size, size=num_samples)
# 답은 정수가 아닌 원-핫 인코딩된 벡터
answers = to_categorical(answers)

# 리스트 입력을 사용하여 학습
model.fit([text, question], answers, epochs=10, batch_size=128)

# model.fit({'text' : text, 'question' : question}, answers, epochs=10, batch_size=128)

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.2145 - acc: 0.0010
Epoch 2/10
1000/1000 [==============================] - 1s 518us/step - loss: 6.1977 - acc: 0.0490
Epoch 3/10
1000/1000 [==============================] - 0s 466us/step - loss: 6.1580 - acc: 0.0270
Epoch 4/10
1000/1000 [==============================] - 0s 463us/step - loss: 6.0533 - acc: 0.0030
Epoch 5/10
1000/1000 [==============================] - 0s 460us/step - loss: 5.9919 - acc: 0.0050
Epoch 6/10
1000/1000 [==============================] - 0s 469us/step - loss: 5.8978 - acc: 0.0080
Epoch 7/10
1000/1000 [==============================] - 0s 463us/step - loss: 5.7944 - acc: 0.0160
Epoch 8/10
1000/1000 [==============================] - 0s 474us/step - loss: 5.7109 - acc: 0.0320
Epoch 9/10
1000/1000 [==============================] - 0s 489us/step - loss: 5.6324 - acc: 0.0430
Epoch 10/10
1000/1000 [==============================] - 0s 485us/step - loss: 5.5466 - acc: 0.0500


#### 7.1.3 다중 출력 모델

다중 출력 모델을 만들어 보자
- 소셜 미디어에서 익명 사용자의 포스트를 입력으로 받아 그 사람의 나이, 성별, 소득 수준 등을 예측

In [16]:
from keras.models import Model
from keras import layers
from keras import Input

voca_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None, ), dtype='int32', name='posts')
embedded_posts = layers.Embedding(voca_size, 256)(posts_input)

x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)

x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)

x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dense(128, activation='relu')(x)

# 출력 중에 이름을 지정
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, None, 256)    12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 128)    163968      embedding_8[0][0]                
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, None, 128)    0           conv1d_1[0][0]                   
____________________________________________________________________________________________

모델을 훈련하려면 네트워크 출력마다 다른 손실 함수를 지정해야함
- 손실 값을 합치는 가장 간단한 방법은 모두 더하는 것
- compile 메소드를 이용해서 처리

In [17]:
model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])
# model.compile(optimizer='rmsprop', loss={'age' : 'mse', 'income' : 'categorical_crossentropy', 'gender' : 'binary_crossentropy'})

손실 값이 많이 불균형하면 모델이 개별 손실이 가장 큰 작업에 치우쳐 표현을 최적화 함
- 다른 작업들은 손해를 입음
- 이를 해결하기 위해 손실 값이 최종 손실에 기여하는 수준을 지정할 수 있음
  - 손실 값의 스케일이 다를 때 유용
- mse는 일반적으로 3~5사이의 값을 가짐, 성별 분류는 0.1
  - 각각 0.25, 10의 가중치를 줌

In [18]:
model.compile(
    optimizer='rmsprop', 
    loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
    loss_weight=[0.25, 1., 10.]
)

# model.compile(
#     optimizer='rmsprop', 
#     loss={'age' : 'mse', 'income' : 'categorical_crossentropy', 'gender' : 'binary_crossentropy'},
#     loss_weight={
#         'age':0.25,
#         'income' : 1.,
#         'gender' : 10.
#     }
# )

In [ ]:
# 데이터 있다고 가정하고 짠 코드

# model.fit(posts, [age_targets, income_targets, gender_targets], epochs=10, batch_size=64)

#### 7.1.4 층으로 구성된 비순환 유향 그래프(DAG)

함수형 API를 사용하면 다중 입력이나 다중 출력 모델뿐만 아니라 내부 토폴로지가 복잡한 네트워크도 만들 수 있다.
- 비순환 유향 그래프(DAG)
- 비순환이라는 것이 중요
  - 다시 말해 이 그래프는 원형을 띨 수 없음 -> 텐서 x가 자기 자신을 출력하는 층의 입력이 될 수 없다.
  - 만들 수 있는 루프(순환 연결)는 순환 층의 내부에 있는 것뿐

가장 유명한 2개는 인셉션 모듈과 잔차 연결

#### 인셉션 모듈

Inception은 합성곱 신경망에서 인기 있는 네트워크 구조
- 네트워크 안의 네트워크라는 구조에 영감을 받은 구조
- 가장 기본적인 인셉션 모듈 형태는 3~4개의 가지를 가짐
- 네트워크가 따로따로 공간 특성과 채널 방향의 특성을 학습하도록 도움
- 한꺼번에 학습하는 것보다 효과가 더 높음
  - 더 복잡한 인셉션은 풀링 연산, 여러가지 합성곱 사이즈, 공간 합성곱이 없는 가지 등등

![alt text](https://images.deepai.org/django-summernote/2019-06-18/2cec735b-2347-4ded-ae2b-e8a8384f7b46.png)

In [ ]:
# 4D 텐서가 입력된다고 가정

from keras import layers

# 모든 가지는 동일한 스트라이드(2)를 사용. 출력 크기를 동일하게 만들어 하나로 합치기 위함
branch_a = layers.Conv2D(128, 1, activation='relu', strides=2)(x)

branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)

branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)

# axis=-1은 현재 배열의 마지막 axis를 의미합니다.
# http://taewan.kim/post/numpy_cheat_sheet/

output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

In [21]:
import keras

In [22]:
model = keras.applications.inception_v3.InceptionV3()

96116736/96112376 [==============================] - 20s 0us/step


In [24]:
# model.summary()